In [ ]:
pip install torch transformers tqdm

In [16]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.pipeline import Pipeline
from tqdm import tqdm
import os

In [ ]:
# def verify_cuda_setup():
#     """
#     Verify CUDA setup and provide detailed information
#     """
#     print("\n=== CUDA Setup Verification ===")
    
#     # Check PyTorch installation
#     print(f"PyTorch Version: {torch.__version__}")
#     print(f"CUDA available: {torch.cuda.is_available()}")
    
#     if torch.cuda.is_available():
#         # Get CUDA version
#         cuda_version = torch.version.cuda
#         print(f"CUDA Version: {cuda_version}")
        
#         # GPU Device information
#         current_device = torch.cuda.current_device()
#         print(f"Current CUDA device index: {current_device}")
#         print(f"GPU Device: {torch.cuda.get_device_name(current_device)}")
        
#         # Memory information
#         print("\nGPU Memory Information:")
#         print(f"Total GPU Memory: {torch.cuda.get_device_properties(current_device).total_memory / 1024**3:.2f} GB")
#         print(f"Memory Allocated: {torch.cuda.memory_allocated(current_device) / 1024**3:.2f} GB")
#         print(f"Memory Cached: {torch.cuda.memory_reserved(current_device) / 1024**3:.2f} GB")
        
#         # CUDA architecture
#         print(f"\nCUDA Architecture:")
#         print(f"Device Capability: {torch.cuda.get_device_capability(current_device)}")
        
#         # Test CUDA computation
#         print("\nTesting CUDA computation...")
#         try:
#             test_tensor = torch.tensor([1.0, 2.0, 3.0], device='cuda')
#             result = test_tensor.sum()
#             print("CUDA computation test: Successful")
#         except Exception as e:
#             print(f"CUDA computation test failed: {e}")
            
#         return torch.device('cuda')
#     else:
#         print("\nWARNING: CUDA is not available. Check your PyTorch installation and NVIDIA drivers.")
#         print("\nDebug Information:")
#         print(f"PyTorch CUDA arch list: {os.environ.get('TORCH_CUDA_ARCH_LIST', 'Not Set')}")
#         print(f"CUDA_HOME: {os.environ.get('CUDA_HOME', 'Not Set')}")
#         print(f"CUDA_PATH: {os.environ.get('CUDA_PATH', 'Not Set')}")
#         return torch.device('cpu')


=== CUDA Setup Verification ===
PyTorch Version: 2.5.1+cu121
CUDA available: True
CUDA Version: 12.1
Current CUDA device index: 0
GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU

GPU Memory Information:
Total GPU Memory: 4.00 GB
Memory Allocated: 0.00 GB
Memory Cached: 0.00 GB

CUDA Architecture:
Device Capability: (8, 6)

Testing CUDA computation...
CUDA computation test: Successful


device(type='cuda')

In [17]:
def verify_cuda_setup():
    """
    Verify CUDA setup and provide detailed information
    """
    print("\n=== CUDA Setup Verification ===")
    
    # Check PyTorch installation
    print(f"PyTorch Version: {torch.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    
    if torch.cuda.is_available():
        # Get CUDA version
        cuda_version = torch.version.cuda
        print(f"CUDA Version: {cuda_version}")
        
        # GPU Device information
        current_device = torch.cuda.current_device()
        print(f"Current CUDA device index: {current_device}")
        print(f"GPU Device: {torch.cuda.get_device_name(current_device)}")
        
        # Memory information
        print("\nGPU Memory Information:")
        print(f"Total GPU Memory: {torch.cuda.get_device_properties(current_device).total_memory / 1024**3:.2f} GB")
        print(f"Memory Allocated: {torch.cuda.memory_allocated(current_device) / 1024**3:.2f} GB")
        print(f"Memory Cached: {torch.cuda.memory_reserved(current_device) / 1024**3:.2f} GB")
        
        # CUDA architecture
        print(f"\nCUDA Architecture:")
        print(f"Device Capability: {torch.cuda.get_device_capability(current_device)}")
        
        # Test CUDA computation
        print("\nTesting CUDA computation...")
        try:
            test_tensor = torch.tensor([1.0, 2.0, 3.0], device='cuda')
            result = test_tensor.sum()
            print("CUDA computation test: Successful")
        except Exception as e:
            print(f"CUDA computation test failed: {e}")
            
        return torch.device('cuda')
    else:
        print("\nWARNING: CUDA is not available. Check your PyTorch installation and NVIDIA drivers.")
        print("\nDebug Information:")
        print(f"PyTorch CUDA arch list: {os.environ.get('TORCH_CUDA_ARCH_LIST', 'Not Set')}")
        print(f"CUDA_HOME: {os.environ.get('CUDA_HOME', 'Not Set')}")
        print(f"CUDA_PATH: {os.environ.get('CUDA_PATH', 'Not Set')}")
        return torch.device('cpu')

In [5]:
verify_cuda_setup()


=== CUDA Setup Verification ===
PyTorch Version: 2.5.1+cu121
CUDA available: True
CUDA Version: 12.1
Current CUDA device index: 0
GPU Device: NVIDIA GeForce RTX 3050 Laptop GPU

GPU Memory Information:
Total GPU Memory: 4.00 GB
Memory Allocated: 0.00 GB
Memory Cached: 0.00 GB

CUDA Architecture:
Device Capability: (8, 6)

Testing CUDA computation...
CUDA computation test: Successful


device(type='cuda')

In [18]:
import nltk

# Download required NLTK data
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [19]:
def load_and_preprocess_data():
    """Load and preprocess the Twitter dataset"""
    column_names = ['target', 'ids', 'date', 'flag', 'user', 'text']
    data = pd.read_csv("twitter_dataset.csv", names=column_names, encoding='ISO-8859-1')
    data['target'] = data['target'].replace(4, 1)
    return data

In [20]:
data = load_and_preprocess_data()
data = data.sample(n=30000, random_state=42)
data.head()
data.shape

(30000, 6)

In [21]:
data.head()

,target,ids,date,flag,user,text
541200,0,2200003196,Tue Jun 16 18:18:12 PDT 2009,NO_QUERY,LaLaLindsey0609,@chrishasboobs AHHH I HOPE YOUR OK!!!
750,0,1467998485,Mon Apr 06 23:11:14 PDT 2009,NO_QUERY,sexygrneyes,"@misstoriblack cool , i have no tweet apps fo..."
766711,0,2300048954,Tue Jun 23 13:40:11 PDT 2009,NO_QUERY,sammydearr,@TiannaChaos i know just family drama. its la...
285055,0,1993474027,Mon Jun 01 10:26:07 PDT 2009,NO_QUERY,Lamb_Leanne,School email won't open and I have geography ...
705995,0,2256550904,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,yogicerdito,upper airways problem


In [24]:
import re

In [25]:
def advanced_text_preprocessing(content):

    # Initialize lemmatizer

    lemmatizer = WordNetLemmatizer()
    # Convert to lowercase and remove special characters
    lemmatized_content = re.sub(r'http\S+|www\S+|https\S+', '', content, flags=re.MULTILINE)  # Remove URLs
    lemmatized_content = re.sub(r'@\w+|\#', '', lemmatized_content)  # Remove mentions and hashtag symbols
    lemmatized_content = re.sub(r'[^a-zA-Z\s]', '', lemmatized_content)  # Remove numbers and punctuation
    lemmatized_content = lemmatized_content.lower()

    # Tokenization
    words = word_tokenize(lemmatized_content)
    
    # Remove stopwords and lemmatize
    stop_words = set(stopwords.words('english'))
    # Add custom stopwords that might not be useful for sentiment
    custom_stopwords = {'rt', 'via'}
    stop_words.update(custom_stopwords)
    processed_words = [
        lemmatizer.lemmatize(word) for word in words if word not in stop_words and len(word) > 2
    ]
    lemmatized_content = ' '.join(processed_words)
    return lemmatized_content

In [26]:
 # Apply advanced text preprocessing
    
print("Applying text preprocessing...")
    
data['lemmatized_content'] = data['text'].apply(advanced_text_preprocessing)

Applying text preprocessing...


In [27]:
class TwitterDataset(Dataset):
    def __init__(self, texts, targets, tokenizer, max_len=128):
        self.texts = texts
        self.targets = targets
        self.tokenizer = tokenizer
        self.max_len = max_len
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = self.targets[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(target, dtype=torch.long)
        }

In [28]:
def train_bert_model(data, epochs=3, batch_size=32, learning_rate=2e-5):
    """
    Train a BERT model for sentiment analysis
    
    Parameters:
    data: pandas DataFrame with 'lemmatized_content' and 'target' columns
    epochs: number of training epochs
    batch_size: batch size for training
    learning_rate: learning rate for optimization
    """
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")

    # Initialize tokenizer and model
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    model = BertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=2
    ).to(device)

    # Prepare data
    X = data['lemmatized_content'].values
    y = data['target'].values
    
    X_train, X_val, y_train, y_val = train_test_split(
        X, y, test_size=0.2, stratify=y, random_state=42
    )

    # Create datasets
    train_dataset = TwitterDataset(X_train, y_train, tokenizer)
    val_dataset = TwitterDataset(X_val, y_val, tokenizer)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)

    # Initialize optimizer
    optimizer = AdamW(model.parameters(), lr=learning_rate)

    # Training loop
    best_accuracy = 0
    for epoch in range(epochs):
        print(f'\nEpoch {epoch + 1}/{epochs}')
        
        # Training phase
        model.train()
        train_losses = []
        train_preds = []
        train_true = []

        for batch in tqdm(train_loader, desc='Training'):
            optimizer.zero_grad()
            
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            targets = batch['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask,
                labels=targets
            )

            loss = outputs.loss
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
            
            preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
            train_preds.extend(preds)
            train_true.extend(targets.cpu().numpy())

        train_accuracy = accuracy_score(train_true, train_preds)
        
        # Validation phase
        model.eval()
        val_losses = []
        val_preds = []
        val_true = []

        with torch.no_grad():
            for batch in tqdm(val_loader, desc='Validation'):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                targets = batch['targets'].to(device)

                outputs = model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    labels=targets
                )

                val_losses.append(outputs.loss.item())
                
                preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
                val_preds.extend(preds)
                val_true.extend(targets.cpu().numpy())

        val_accuracy = accuracy_score(val_true, val_preds)
        
        print(f'\nTraining Loss: {np.mean(train_losses):.4f}')
        print(f'Training Accuracy: {train_accuracy:.4f}')
        print(f'Validation Loss: {np.mean(val_losses):.4f}')
        print(f'Validation Accuracy: {val_accuracy:.4f}')
        
        if val_accuracy > best_accuracy:
            best_accuracy = val_accuracy
            print("\nSaving best model...")
            torch.save(model.state_dict(), 'best_bert_model.pt')
            
        print('\nClassification Report:')
        print(classification_report(val_true, val_preds))

    return model, tokenizer

In [29]:
def predict_sentiment(text, model, tokenizer, device=None):
    """
    Predict sentiment for a single text input with CUDA optimization
    """
    if device is None:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    model.eval()
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=128,
        padding='max_length',
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        if device.type == 'cuda':
            with torch.cuda.amp.autocast():
                outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        else:
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        prediction = torch.argmax(outputs.logits, dim=1).item()
    
    return "Positive" if prediction == 1 else "Negative"

(30000, 6)

In [30]:
model, tokenizer = train_bert_model(data)

Using device: cuda


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\nikhi\AppData\Roaming\Python\Python312\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(



Epoch 1/3


Validation: 100%|██████████| 188/188 [00:41<00:00,  4.58it/s]



Training Loss: 0.5130
Training Accuracy: 0.7440
Validation Loss: 0.4911
Validation Accuracy: 0.7715

Saving best model...

Classification Report:
              precision    recall  f1-score   support

           0       0.80      0.73      0.76      3000
           1       0.75      0.82      0.78      3000

    accuracy                           0.77      6000
   macro avg       0.77      0.77      0.77      6000
weighted avg       0.77      0.77      0.77      6000


Epoch 2/3


Validation: 100%|██████████| 188/188 [00:41<00:00,  4.58it/s]



Training Loss: 0.4067
Training Accuracy: 0.8170
Validation Loss: 0.4825
Validation Accuracy: 0.7775

Saving best model...

Classification Report:
              precision    recall  f1-score   support

           0       0.79      0.76      0.77      3000
           1       0.77      0.80      0.78      3000

    accuracy                           0.78      6000
   macro avg       0.78      0.78      0.78      6000
weighted avg       0.78      0.78      0.78      6000


Epoch 3/3


Validation: 100%|██████████| 188/188 [00:41<00:00,  4.56it/s]


Training Loss: 0.2952
Training Accuracy: 0.8764
Validation Loss: 0.6069
Validation Accuracy: 0.7687

Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.74      0.76      3000
           1       0.76      0.80      0.77      3000

    accuracy                           0.77      6000
   macro avg       0.77      0.77      0.77      6000
weighted avg       0.77      0.77      0.77      6000



In [31]:
test_text = "This is a wonderful day! I'm really happy!"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(f"\nTest text: {test_text}")
print(f"Predicted sentiment: {sentiment}")


Test text: This is a wonderful day! I'm really happy!
Predicted sentiment: Positive


C:\Users\nikhi\AppData\Local\Temp\ipykernel_7592\3602477452.py:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


In [ ]:
torch.cuda.is_available()


In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.get_device_name(0)

# BERT Sentiment Analysis Project Report

## Project Overview
This project implements a sentiment analysis classifier using BERT (Bidirectional Encoder Representations from Transformers) on a Twitter dataset to predict tweet sentiment.

## Model Architecture
- **Base Model**: BERT-base-uncased
- **Classification Type**: Binary Sentiment Classification
- **Input Preprocessing**: Advanced text lemmatization and cleaning

## Dataset Characteristics
- **Total Samples**: 30,000 tweets
- **Training Split**: 24,000 tweets
- **Validation Split**: 6,000 tweets
- **Class Distribution**: Balanced (50% positive, 50% negative)

## Training Configuration
- **Training Epochs**: 3
- **Batch Size**: 32
- **Learning Rate**: 2e-5
- **Device**: CUDA (GPU-accelerated)

## Performance Metrics

### Epoch-wise Performance
| Epoch | Training Loss | Training Accuracy | Validation Loss | Validation Accuracy |
|-------|--------------|-------------------|-----------------|---------------------|
| 1     | 0.5130       | 0.7440            | 0.4911          | 0.7715              |
| 2     | 0.4067       | 0.8170            | 0.4825          | 0.7775              |
| 3     | 0.2952       | 0.8764            | 0.6069          | 0.7687              |

### Final Model Performance
| Metric       | Value |
|--------------|-------|
| Accuracy     | 0.7775 (77.75%) |
| Precision (Negative) | 0.79 |
| Precision (Positive) | 0.77 |
| Recall (Negative)    | 0.76 |
| Recall (Positive)    | 0.80 |
| F1-Score (Negative)  | 0.77 |
| F1-Score (Positive)  | 0.78 |

## Key Observations
1. **Model Learning**: 
   - Training accuracy increases with each epoch
   - Validation accuracy remains relatively stable
2. **Generalization**: Moderate overfitting observed
3. **Performance Consistency**: Balanced performance across both sentiment classes

## Preprocessing Techniques
- URL removal
- Mention and hashtag symbol removal
- Lowercase conversion
- Lemmatization
- Stopword elimination
- Custom stopword addition

## Technical Stack
- **Deep Learning Framework**: PyTorch
- **Transformer Library**: Hugging Face Transformers
- **Preprocessing**: NLTK
- **Model**: BERT-base-uncased

## Computational Resources
- **Device**: CUDA-enabled GPU
- **GPU**: Detected and utilized for training

## Recommendations for Future Improvement
1. Experiment with larger BERT variants (e.g., RoBERTa, BERT-large)
2. Implement data augmentation techniques
3. Use learning rate scheduling
4. Explore ensemble methods
5. Collect more diverse training data

## Sample Prediction
- **Test Text**: "This is a wonderful day! I'm really happy!"
- **Predicted Sentiment**: Positive

## Conclusion
The BERT-based sentiment analysis model provides a robust solution for classifying tweet sentiments, achieving nearly 78% accuracy with sophisticated deep learning techniques.

**Date of Analysis**: November 28, 2024
**Dataset**: Twitter Sentiment Dataset
**Total Processed Samples**: 30,000 tweets